# Lectura de datos avro
Primero que todo, como nos interesa los datos de eventos transitorios, nos encontramos con una pagina web (https://ztf.uw.edu/alerts/public/) la cual es un telescopio que mapea cada día el cielo nocturno para obtener imagenes de eventos transitorios. Estos datos vienen en formato avro los cuales son archivos que almacenan datos en formato binario utilizando el esquema de los datos en un archivo contenedor.

### Caracteristicas clave de los archivos avro
- Serialización de datos:
Avro serializa datos en un formato binario compacto y eficiente. 
- Esquemas definidos:
Los datos están asociados con un esquema definido en formato JSON, lo que facilita la lectura y el procesamiento de los datos. 
- Evolución del esquema:
Avro permite la evolución de los esquemas sin problemas, lo que significa que los programas antiguos pueden leer datos nuevos y viceversa. 
- Compatibilidad con diferentes lenguajes:
Avro ofrece API para varias plataformas, incluyendo Java, Python, Ruby, C, C++ y más. 
- Integración con Hadoop:
Avro es ampliamente utilizado en el ecosistema Hadoop para el almacenamiento y procesamiento de datos. 
- Ideal para streaming:
Avro es adecuado para la transmisión de datos entre sistemas, ya que serializa los datos de manera independiente por filas. 
### Uso de archivos Avro:
- Almacenamiento de datos:
Avro es un formato de archivo útil para almacenar datos de forma persistente en sistemas de almacenamiento distribuido. 
- Transferencia de datos:
Avro facilita la transferencia de datos entre sistemas y aplicaciones, especialmente en entornos de streaming. 
- Procesamiento de datos:
Avro puede ser utilizado para el procesamiento de datos en paralelo, aprovechando las capacidades de los frameworks de computación distribuida como Apache Hadoop. 
- Integración con Kafka:
Apache Kafka utiliza Avro para la serialización de mensajes, lo que permite la transmisión eficiente de datos. 

#### Información importante 
en nuestro caso nos benefició el formato avro ya que al compactarlos en formato binario, lo que nos entregan los archivo es un cubo 3d de imagenes 21x21 en el cual también se traen los metadatos, por lo tanto como nos interesa entrenar una red neuronal convolucional decidimos solamente extraer los datos de las imagenes

In [ ]:
import fastavro
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
import gzip
from astropy.io import fits
import warnings
import pandas as pd
from astropy.io.fits.verify import VerifyWarning
import os 
import glob

def store_ztf_stamps_3d(input_dir, output_dir="ztf_3d_stamps", crop_to_21x21=True):
    """
    Procesa archivos Avro de ZTF en un directorio, extrae los stamps (science, reference, difference),
    y los almacena en una matriz tridimensional (3, 21, 21) por alerta.

    Parameteros:
    - input_dir (str): Directorio con los archivos Avro.
    - output_dir (str): Directorio donde se guardarán las matrices tridimensionales (.npy).
    - crop_to_21x21 (bool): Si True, recorta los stamps a 21x21 píxeles (como en ALeRCE).
    """
    # Crear directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    # Ignorar advertencias de verificación de FITS
    warnings.filterwarnings('ignore', category=VerifyWarning)

    # Encontrar todos los archivos Avro en el directorio de entrada
    avro_files = glob.glob(os.path.join(input_dir, "*.avro"))
    if not avro_files:
        print(f"No se encontraron archivos Avro en {input_dir}")
        return

    print(f"Procesando {len(avro_files)} archivos Avro en {input_dir}")

    # Procesar cada archivo Avro
    for i, avro_file in enumerate(avro_files, 1):
        print(f"\nLeyendo archivo {i}/{len(avro_files)}: {avro_file}")
        try:
            with open(avro_file, 'rb') as f:
                reader = fastavro.reader(f)
                # Iterar sobre las alertas en el archivo
                for alert in reader:
                    # Extraer ID de la alerta
                    object_id = alert.get('objectId', 'Unknown')
                    print(f"Procesando Alerta ID: {object_id}")

                    # Extraer imágenes (stamps)
                    science_stamp = alert.get('cutoutScience', {}).get('stampData', None)
                    reference_stamp = alert.get('cutoutTemplate', {}).get('stampData', None)
                    difference_stamp = alert.get('cutoutDifference', {}).get('stampData', None)

                    # Función para decodificar un stamp
                    def decode_stamp(stamp_data):
                        '''esta función se encarga de descomprimir el stamp, lo lee como un archivo FITS
                        verifica tamaño para luego recotarlo en 21x21 reemplaza los NaN con 0 y devuelve
                        el arreglo de imagen'''
                        if stamp_data:
                            try:
                                # Descomprimir datos gzip
                                decompressed_data = gzip.decompress(stamp_data)
                                # Leer datos FITS desde BytesIO
                                with fits.open(BytesIO(decompressed_data), ignore_missing_simple=True) as hdu_list:
                                    stamp_array = hdu_list[0].data.astype(np.float32)
                                # Verificar tamaño esperado (63x63 para ZTF)
                                if stamp_array.shape != (63, 63):
                                    print(f"Advertencia: El stamp tiene tamaño {stamp_array.shape}, esperado (63, 63)")
                                    return None
                                # Recortar a 21x21 píxeles si se solicita
                                if crop_to_21x21:
                                    stamp_array = stamp_array[21:42, 21:42]
                                # Reemplazar NaN por 0
                                stamp_array = np.nan_to_num(stamp_array, nan=0.0)
                                return stamp_array
                            except Exception as e:
                                print(f"Error al decodificar stamp: {e}")
                                return None
                        return None

                    # Decodificar los stamps
                    science_array = decode_stamp(science_stamp)
                    reference_array = decode_stamp(reference_stamp)
                    difference_array = decode_stamp(difference_stamp)

                    # Verificar que todos los stamps se cargaron correctamente
                    if science_array is not None and reference_array is not None and difference_array is not None:
                        # Crear matriz tridimensional (3, 21, 21) o (3, 63, 63) según crop_to_21x21
                        stamp_size = 21 if crop_to_21x21 else 63
                        stamps_3d = np.zeros((3, stamp_size, stamp_size), dtype=np.float32)
                        stamps_3d[0] = science_array
                        stamps_3d[1] = reference_array
                        stamps_3d[2] = difference_array

                        # Guardar la matriz tridimensional como archivo .npy
                        output_path = os.path.join(output_dir, f'alert_{object_id}_stamps_3d.npy')
                        np.save(output_path, stamps_3d)
                        print(f"Matriz 3D guardada en: {output_path} (forma: {stamps_3d.shape})")
                    else:
                        print(f"No se pudieron cargar todos los stamps para la alerta {object_id}")
        except Exception as e:
            print(f"Error al procesar el archivo {avro_file}: {e}")
# Ejemplo de uso

if __name__ == "__main__":
    input_directory = "/home/seba/Usach/bigdata/Proyecto/datos4"  # Reemplaza con la ruta al directorio con archivos Avro
    store_ztf_stamps_3d(input_directory, output_dir="/home/seba/Usach/bigdata/Proyecto/ztf_3d_stamps_4", crop_to_21x21=True)

# datos etiquetados 
Como nos interesa cerar una CNN para clasificar eventos transitorios necesitamos de datos etiquetados, los cuales pudimos encontrar en  https://zenodo.org/records/4279623 para luego realizar un crossmatch con nuestros datos a partir del id  

In [ ]:

alerce_data = pd.read_csv("/home/seba/Usach/bigdata/Proyecto/etiquetas/ALeRCE_lc_classifier_outputs_ZTF_unlabeled_set_20200609.csv")

In [5]:
# Muestra las etiquetas únicas y su número total en alerce_data
print("\nEtiquetas en alerce_data['predicted_class']:")
print(alerce_data['predicted_class'].value_counts())


Etiquetas en alerce_data['predicted_class']:
predicted_class
Periodic-Other    243374
E                 198122
LPV               161592
YSO                85087
RRL                58592
QSO                43054
DSCT               26672
CEP                17307
AGN                14342
CV/Nova             7945
Blazar              5085
SNIa                3956
SNIbc               1626
SNII                 890
SLSN                 727
Name: count, dtype: int64


#### Data sets
Nos interesa obtener grandes volumenes de datos para el entrenamiento, por lo tanto realizamos un codigo que se encarga de sacar variaos set de datos y juntarlos en un puro dataframe, vale recalcar que los datos son los extraidos anteriormente en binario (formato .npy)

In [ ]:

folders = [
    "/home/seba/Usach/bigdata/Proyecto/ztf_3d_stamps",
    "/home/seba/Usach/bigdata/Proyecto/ztf_3d_stamps_2",
    "/home/seba/Usach/bigdata/Proyecto/ztf_3d_stamps_3",
    "/home/seba/Usach/bigdata/Proyecto/ztf_3d_stamps_4"
]

rows = []

for i, folder in enumerate(folders, 1):
    file_list = glob.glob(os.path.join(folder, "alert_*_stamps_3d.npy"))
    total_files = len(file_list)
    for j, filepath in enumerate(file_list, 1):
        filename = os.path.basename(filepath)
        object_id = filename.split('_')[1]
        data = np.load(filepath)
        rows.append({'object_id': object_id, 'data': data})
        print(f'Archivos procesados en carpeta {i}: {j} de {total_files}', end='\r')


avro_data = pd.DataFrame(rows) # crear un dataframe con columnas: object_id, data
avro_data = avro_data.drop_duplicates(subset='object_id', keep='first') # eliminamos duplicados con el mismo object_id

print(avro_data.shape)
print(avro_data.head())

(358163, 2)ocesados en carpeta 4: 147156 de 147156
      object_id                                               data
0  ZTF25aapoqyu  [[[291.5396, 300.09894, 299.94568, 299.8415, 3...
1  ZTF21aayefhf  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
2  ZTF21acnhdfa  [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 250...
3  ZTF19acsjkbr  [[[234.52643, 238.25146, 243.91174, 239.44464,...
4  ZTF21aamjywk  [[[286.34802, 323.42926, 302.9098, 286.79755, ...


# Cross Match
ahora que tenemos el set de datos toca realizar el crossmatch, esto es importante ya que la red neuronal debe de saber que clase de evento transitorio está analizando (en nuestro caso nos decantamos por QSO, YSO y AGN, también añadimos "otros"), ademas de que el set de datos etiquetados no tiene imagenes y por lo tanto es completamente necesario realizar el crossmatch

In [ ]:
# Crear una nueva columna de etiqueta, asignando 'Otro' a los que no son QSO, YSO o AGN
alerce_data['label'] = alerce_data['predicted_class'].where(
    alerce_data['predicted_class'].isin(['QSO', 'YSO', 'AGN']),
    other='Otro'
)

# Seleccionar solo las columnas necesarias y renombrar para el merge
alerce_labels = alerce_data[['oid', 'label']].rename(columns={'oid': 'object_id'})

# Hacer el cross-match con avro_data
final_avro_df = pd.merge(avro_data, alerce_labels, on='object_id', how='inner')

print(final_avro_df['label'].value_counts())
print(final_avro_df.head())

label
Otro    122968
YSO       6935
QSO       2804
AGN       1359
Name: count, dtype: int64
      object_id                                               data label
0  ZTF18acvwapg  [[[284.2307, 270.86517, 263.56464, 281.25702, ...  Otro
1  ZTF18adalcju  [[[366.73608, 359.24762, 350.893, 348.76733, 3...  Otro
2  ZTF19aahibom  [[[337.14615, 369.51688, 363.7308, 388.83627, ...  Otro
3  ZTF18adbzbnk  [[[295.16556, 283.30942, 293.73746, 276.49637,...  Otro
4  ZTF19aaskrrm  [[[349.99973, 350.7228, 340.42014, 354.07413, ...  Otro


#### Guardar Dataset en varios csv
en nuestro caso nos acomoda guardar los datos en csv, debido a que sabemos manejar este tipo de datos  

In [ ]:
# filtramos por cada clase y tomamos una muestra eleatoria de 12000 ejemplos de la clase 'otro' para balancear el dataset
# finalmente como nos indico nuestro especialista en redes neuronales, guardaremos cada clase en un csv separado
Otro = final_avro_df[final_avro_df['label'] == 'Otro']
Otro = Otro.sample(n=12000)

Quasar = final_avro_df[final_avro_df['label'] == 'QSO']
AGN = final_avro_df[final_avro_df['label'] == 'AGN']
YSO = final_avro_df[final_avro_df['label'] == 'YSO']


Quasar.to_csv("ztf_avro_Grande_QSO.csv", index=False)
AGN.to_csv("ztf_avro_Grande_AGN.csv", index=False)
YSO.to_csv("ztf_avro_Grande_YSO.csv", index=False)
Otro.to_csv("ztf_avro_Grande_Otro.csv", index=False)

# En resumen
### el codigo lo que hace es:
- procesar los datos de alerta ZTF desde archivos avro
- extraer las imagenes en formato 3D (3 canales) para cada alerta (también se limpian los datos transformando los NaN a 0)
- unimos los datos con etiquetas entregadas por alerce (hacemos un crossmatch con un dataset pequeño el cual tiene datos etiquetados)
- filtramos por clases de nuestro interes y agregamos una nueva clase para evitar error de clasificación (QSO,YSO,AGN,Otro)
- exportamos el conjunto de datos etiquetados en CSV para futuros entrenamientos de la CNN 